In [1]:
# %load honest_net.py
import numpy as np
import pandas as pd

df = pd.read_csv("../../data/honest_net.tsv", sep="\t")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]


# C&P from https://stackoverflow.com/a/39513799
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad / np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g


def parse_array(s):
    try:
        return np.fromstring(s, dtype=float, sep="|")
    except TypeError:
        return np.array([float("nan")])


def expand(row):
    compute = parse_array(row.compute)
    weakest = np.argmin(compute)
    strongest = np.argmax(compute)
    d = {}

    def wsg(k, v):
        d[k + "_weakest"] = v[weakest]
        d[k + "_strongest"] = v[strongest]
        d[k + "_gini"] = gini(v)

    rcompute = compute / np.sum(compute)
    wsg("compute", rcompute)
    activations = parse_array(row.activations)
    assert np.sum(activations) == row.number_activations or row.error
    ractivations = activations / np.sum(activations)
    wsg("activations", ractivations)
    reward = parse_array(row.reward)
    rreward = reward / np.sum(reward)
    wsg("reward", rreward)
    efficiency = rreward / ractivations
    wsg("efficiency", efficiency)
    d["activations_compute_gini_delta"] = d["activations_gini"] - d["compute_gini"]
    d["reward_activations_gini_delta"] = d["reward_gini"] - d["activations_gini"]
    return d


df = df.join(df.apply(expand, axis=1, result_type="expand"))

## Incentive Schemes for Georges  Protocol

In [2]:
d = df[df.protocol == "george"]

### Efficiency of the weakest miner

We pick the weakest miner by compute power and calculate its

efficiency = relative reward / relative activations

In [6]:
eff = d.rename(
    columns={'incentive_scheme': 'Incentive Scheme',
            'block_interval': 'Block Interval'}
).pivot(
    index=["Block Interval", "k"],
    columns=["Incentive Scheme"],
    values="efficiency_weakest",
)
eff.to_latex(buf="../../tex/george-reward-inequality/tab/efficiency-weakest.tex")
eff

Incentive Scheme       block  constant  discount    hybrid    punish
Block Interval k                                                    
30.0           1    0.999442  0.999442  0.999442  0.999442  0.999442
               2    0.979606  0.985236  0.985236  0.985236  0.985236
               4    0.915076  0.941221  0.944049  0.924185  0.916485
               8    0.890585  0.996607  0.984788  0.944924  0.954946
               16   0.384335  0.990862  0.970742  0.777941  0.792773
               32   0.180923  0.978117  0.973145  0.750619  0.740089
               64   1.202887  1.008671  0.997649  0.423890  0.443496
               128  0.734862  0.987471  0.988059  0.265202  0.267984
60.0           1    1.012146  1.012146  1.012146  1.012146  1.012146
               2    1.056290  0.993416  0.993416  0.993416  0.993416
               4    0.927672  0.984084  0.982442  0.981157  0.982442
               8    1.094691  1.008535  1.012543  0.988433  0.983025
               16   0.439599  0.983603  0.985509  0.912727  0.904197
               32   0.965009  0.971040  0.967340  0.881108  0.876160
               64   0.000000  0.988779  0.983423  0.720893  0.726412
               128  1.515152  0.964725  0.964090  0.483052  0.482988
120.0          1    1.001160  1.001160  1.001160  1.001160  1.001160
               2    1.044023  0.986022  0.986022  0.986022  0.986022
               4    0.881217  0.997489  0.988725  0.984350  0.991825
               8    1.159555  1.009242  1.002317  0.999515  1.004409
               16   0.656858  1.002881  1.002772  0.988927  0.991505
               32   1.240887  0.988832  0.987977  0.953718  0.954489
               64   0.000000  1.000541  0.994925  0.863331  0.867116
               128  0.000000  1.000027  1.005350  0.919616  0.913899
300.0          1    1.003311  1.003311  1.003311  1.003311  1.003311
               2    0.984076  1.000850  1.000850  1.000850  1.000850
               4    0.789366  0.999863  1.004181  1.003715  0.999218
               8    0.813286  0.996275  1.000004  1.005479  1.001941
               16   0.967943  0.995442  0.993763  0.983744  0.983046
               32   1.213371  1.004823  1.004466  0.989109  0.987957
               64   1.046207  0.997167  0.996089  0.949326  0.951678
               128  0.000000  0.976220  0.975965  0.854483  0.850668
600.0          1    1.001201  1.001201  1.001201  1.001201  1.001201
               2    0.950791  1.003613  1.003613  1.003613  1.003613
               4    1.064283  1.002406  1.001379  1.004106  1.006036
               8    1.095525  1.002406  1.004013  1.001764  0.999870
               16   1.152860  1.003210  1.001958  0.995650  0.997002
               32   0.761951  1.004823  1.003192  0.992148  0.993259
               64   1.180173  1.001918  1.005953  0.960504  0.956716
               128  1.352814  1.009326  1.009153  1.020453  1.021242

### Reward Gini Fairness

We compare the rewards of all miners using the Gini coefficient. A lower coefficient implies more fairness. All numbers should be compared to the Gini coefficient of the compute powers.

In [4]:
d.compute_gini.unique()

array([0.3])

In [5]:
gini = d.rename(
    columns={'incentive_scheme': 'Incentive Scheme',
            'block_interval': 'Block Interval'}
).pivot(
    index=["Block Interval", "k"],
    columns=["Incentive Scheme"],
    values="reward_gini",
)
gini.to_latex(buf="../../tex/george-reward-inequality/tab/reward-gini.tex")
gini

Incentive Scheme       block  constant  discount    hybrid    punish
Block Interval k                                                    
30.0           1    0.302911  0.302911  0.302911  0.302911  0.302911
               2    0.308298  0.309081  0.309081  0.309081  0.309081
               4    0.298475  0.298645  0.299486  0.301983  0.301920
               8    0.330365  0.301887  0.303339  0.310085  0.309476
               16   0.319388  0.303784  0.304879  0.323705  0.323368
               32   0.296599  0.302849  0.304909  0.352080  0.351481
               64   0.392414  0.308297  0.310311  0.407951  0.405793
               128  0.461111  0.316862  0.317919  0.506265  0.504052
60.0           1    0.302085  0.302085  0.302085  0.302085  0.302085
               2    0.300990  0.298783  0.298783  0.298783  0.298783
               4    0.324671  0.311040  0.311763  0.313125  0.312805
               8    0.294239  0.301255  0.302210  0.305846  0.305507
               16   0.317851  0.302417  0.302620  0.313031  0.313379
               32   0.360726  0.295813  0.296091  0.314185  0.314643
               64   0.466887  0.309851  0.311064  0.360138  0.359251
               128  0.494667  0.306417  0.306848  0.399512  0.399126
120.0          1    0.301248  0.301248  0.301248  0.301248  0.301248
               2    0.287454  0.295473  0.295473  0.295473  0.295473
               4    0.309102  0.307273  0.307913  0.309051  0.308767
               8    0.288636  0.304586  0.304833  0.307717  0.307889
               16   0.321104  0.305499  0.305846  0.311827  0.311695
               32   0.315635  0.301588  0.302077  0.311518  0.311535
               64   0.387582  0.306618  0.307424  0.334166  0.333595
               128  0.323684  0.295909  0.295701  0.340879  0.341431
300.0          1    0.301425  0.301425  0.301425  0.301425  0.301425
               2    0.299758  0.296659  0.296659  0.296659  0.296659
               4    0.322464  0.305717  0.305675  0.305578  0.305589
               8    0.305314  0.301047  0.301112  0.300980  0.300859
               16   0.321900  0.304147  0.303846  0.305698  0.306150
               32   0.343087  0.303698  0.303721  0.307403  0.307418
               64   0.349032  0.309617  0.309617  0.316975  0.316968
               128  0.403896  0.296043  0.295778  0.315064  0.315629
600.0          1    0.300421  0.300421  0.300421  0.300421  0.300421
               2    0.290767  0.292613  0.292613  0.292613  0.292613
               4    0.316840  0.306496  0.306383  0.306522  0.306680
               8    0.297915  0.303909  0.304102  0.304216  0.304044
               16   0.299679  0.301003  0.300930  0.302004  0.302133
               32   0.344373  0.294956  0.295289  0.296703  0.296458
               64   0.361935  0.304355  0.304242  0.307255  0.307373
               128  0.349351  0.298072  0.298294  0.307077  0.306892